In [27]:
import pandas as pd
import datetime
import requests
from configparser import ConfigParser
import json, requests, datetime
from pysimplicate import Simplicate

# Pandas
pd.options.display.float_format = '{:,.1f}'.format
pd.set_option('display.max_rows', 50)

# Simplicate
ini = ConfigParser()
ini.read('../credentials.ini')

subdomain = ini['simplicate']['subdomain']
api_key = ini['simplicate']['api_key']
api_secret = ini['simplicate']['api_secret']

sim = Simplicate(subdomain, api_key, api_secret )

In [ ]:
projects = sim.project({'active':'True'})
projects[0]

In [ ]:
projects = sorted(projects, key=lambda a:a['organization']['name'])
last_client = ''
client_count = 0
for p in projects:
    #if p['project_manager']['name'] == 'Caspar Geerlings':
    #    continue
    #if p['project_manager']['name'] == 'Angela Duijs':
    #    continue
    if p['updated_at'] > '2021-02-19':
        continue
    if p['organization']['name'] != last_client:
        last_client = p['organization']['name']
        print( last_client )
        client_count +=1
    print( '  ', p['project_number'], p['updated_at'], p['name'], p['project_manager']['name'])


In [ ]:
print( client_count, 'clients')
print( len(projects), 'projects')

## Onderhanden werk

In [28]:
session = requests.Session()
login_url = 'https://oberon.simplicate.com/site/login'
login_data = {
    'LoginForm[username]': ini['simplicate']['username'],
    'LoginForm[password]': ini['simplicate']['password'],
}
report_url = 'https://oberon.simplicate.com/v1/reporting/process/reloadData'
session.post(login_url, login_data)

json_data = session.get(report_url).json()

In [29]:
table = json_data['table']
columns = table['columns']
column_names = [column['title'] for column in columns]
column_names

['Huidige status',
 'Begindatum',
 'Einddatum',
 'Budget',
 'Besteed',
 'Gepland',
 'Correcties',
 'Restant budget',
 ' Budget',
 ' Marge',
 'Verkoopbudget',
 ' Besteed',
 'Afgeboekt',
 'Marge gerealiseerd',
 ' Restant budget',
 'Verkoop gerealiseerd',
 'Bruto marge',
 'Kostprijs mdw.',
 'Projectresult.',
 'Verwacht',
 'Gefactureerd',
 'OHW']

In [30]:
rows = table['rows']
dict_list = []
for row in rows[1:]:
    row_data = row['columns']
    row_values = [row[0]['value'] for row in row_data]
    values_as_list = list(zip(column_names, row_values))
    values_as_dict = {l[0]:l[1] for l in values_as_list}
    values_as_dict['Projectnaam'] = row['headers'][0]['value']
    dict_list += [values_as_dict]
df = pd.DataFrame(dict_list)
df

,Huidige status,Begindatum,Einddatum,Budget,Besteed,Gepland,Correcties,Restant budget,Budget,Marge,...,Marge gerealiseerd,Restant budget,Verkoop gerealiseerd,Bruto marge,Kostprijs mdw.,Projectresult.,Verwacht,Gefactureerd,OHW,Projectnaam
0,Actief,16-12-2020,,0,"222,187.5",0,0.0,0,0,0,...,0.0,0,0.0,"220,482.5",0,220483,222188,"220,482.5",1705,Accell IT - Headless 2021 (ACC-1)
1,Actief,21-04-2021,01-10-2021,0,"14,355.0",0,0.0,0,0,0,...,0.0,0,0.0,"14,085.0",0,14085,14355,"14,085.0",270,Adyen - Marcom platform (ADY-1)
2,Actief,07-03-2019,,9148,"9,172.5",0,-50.0,26,0,0,...,0.0,0,0.0,"8,000.0",0,8000,8000,"8,000.0",-25,Alda Nederland B.V. - ALDA General Strippenkaa...
3,Actief,01-01-2021,,780,601.2,0,0.0,179,0,0,...,0.0,0,0.0,0.0,0,0,780,0.0,601,Alex de Rollende Fietsenmaker - Alex en Iris 2...
4,Actief,04-01-2021,,0,"32,395.0",0,"-12,182.5",0,0,0,...,0.0,0,0.0,"15,895.0",0,15895,20213,"15,895.0",4318,Amsterdam Internet Exchange BV. - AMS-IX SLA (...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,Actief,,,10332,"12,638.8",0,"-1,125.0",-109,0,0,...,0.0,0,0.0,"10,552.5",0,10553,11233,"10,552.5",405,Warner Music Benelux B.V. - Maestro Maintenanc...
105,Actief,01-06-2021,,0,27.5,0,0.0,0,0,0,...,0.0,0,0.0,0.0,0,0,28,0.0,28,Warner Music Benelux B.V. - Maestro Music - SL...
106,Actief,01-01-2021,,0,0.0,0,0.0,0,0,0,...,0.0,0,0.0,"4,310.9",0,4311,0,"4,310.9",0,Weiss Legal - Weiss Legal: Onderhuur 6e etage ...
107,Actief,09-02-2019,,0,0.0,0,0.0,0,0,0,...,0.0,0,0.0,"1,044.0",0,1044,0,"1,044.0",0,Worrell & Jetten Accountants en Adviseurs Zoet...


In [31]:
df['OHW'].sum()

127122

In [39]:
df[['Projectnaam','OHW']].query('abs(OHW)>5000')['OHW'].sum()

123464

In [40]:
df[['Projectnaam','OHW']].query('abs(OHW)>5000')

,Projectnaam,OHW
22,EasyBroker - Design app (easy-2),6120
25,Havas Lemz - FME Platform (HAVA-2),35654
53,Oerol - Platform (OER-1),22375
60,Pure Matching - Pure Matching - SLA (PM-1),5033
67,Sprout Money BV - Marktgevoel Agile (MG2021),7672
68,Sprout Money BV - Slim Beleggen Agile (SM2021),-27669
70,Sprout Money BV - Sprout Money Infrastructure ...,-12963
77,T-Mobile Netherlands B.V. - BEN App (BEN-1),26593
78,T-Mobile Netherlands B.V. - CEO App - SLA (CEO-1),14693
81,ThiemeMeulenhoff B.V. - Examenbundel Release #...,17955
